# DSW export

In [1]:
import glob
import dask.distributed as dsk
import matplotlib.pyplot as plt
from pathlib import Path

import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
import matplotlib.path as mpath
from IPython.display import display, Image

import netCDF4 as nc
import xarray as xr
import numpy as np
import xgcm
import calendar

from matplotlib import gridspec
from collections import OrderedDict

import warnings # ignore these warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

In [ ]:
from dask.distributed import Client
client = Client(threads_per_worker=1)
client

In [ ]:
# Contour mask
contour_mask_1 = xr.open_dataset('/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_01deg.nc')#['contour_masked_above']
contour_mask_2 = xr.open_dataset('/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_005deg.nc')#['contour_masked_above']
contour_mask_3

In [ ]:
lat_slice = slice(-79,-59)
grid_01 = xr.open_dataset('/g/data/ol01/outputs/mom6-panan/panant-01-zstar-ACCESSyr2/output050/20050501.ocean_static.nc')#.sel(yq=lat_slice,yh=lat_slice)
grid_005 = xr.open_dataset('/g/data/ol01/outputs/mom6-panan/panant-005-zstar-ACCESSyr2/output030/19930701.ocean_static.nc')#.sel(yq=lat_slice,yh=lat_slice)
grid_acc = xr.open_dataset(

In [ ]:
# Grids and masks
grids = {'01deg': grid_01, '005deg': grid_005, 'access': grid_acc}
masks = {'01deg': contour_mask_1, '005deg': contour_mask_2, 'access': contour_mask_3}

# Dictionary to store results
contour_data = {}

for res in ['01deg', '005deg', 'access']:
    grid = grids[res]
    contour_mask = masks[res]

    # Select grid coordinates
    yh = grid['yh'].sel(yh=lat_slice)
    yq = grid['yq'].sel(yq=lat_slice)
    xh = grid['xh']
    xq = grid['xq']

    # Select masks for this lat slice
    mask_y_transport = contour_mask['mask_y_transport'].sel(yq=lat_slice)
    mask_x_transport = contour_mask['mask_x_transport'].sel(yh=lat_slice)
    mask_y_transport_numbered = contour_mask['mask_y_transport_numbered'].sel(yq=lat_slice)
    mask_x_transport_numbered = contour_mask['mask_x_transport_numbered'].sel(yh=lat_slice)

    mask_x_transport = xr.DataArray(
        mask_x_transport,
        coords=[yh[:mask_x_transport.shape[0]], xq[:mask_x_transport.shape[1]]],
        dims=['y_ocean', 'x_ocean']
    )
    mask_y_transport = xr.DataArray(
        mask_y_transport,
        coords=[yq[:mask_y_transport.shape[0]], xh[:mask_y_transport.shape[1]]],
        dims=['y_ocean', 'x_ocean']
    )
    mask_x_transport_numbered = xr.DataArray(
        mask_x_transport_numbered,
        coords=[yh[:mask_x_transport_numbered.shape[0]], xq[:mask_x_transport_numbered.shape[1]]],
        dims=['y_ocean', 'x_ocean']
    )
    mask_y_transport_numbered = xr.DataArray(
        mask_y_transport_numbered,
        coords=[yq[:mask_y_transport_numbered.shape[0]], xh[:mask_y_transport_numbered.shape[1]]],
        dims=['y_ocean', 'x_ocean']
    )

    # Stack contour data into 1D
    mask_x_numbered_1d = mask_x_transport_numbered.stack(contour_index=['y_ocean', 'x_ocean'])
    mask_x_numbered_1d = mask_x_numbered_1d.where(mask_x_numbered_1d > 0, drop=True)
    mask_y_numbered_1d = mask_y_transport_numbered.stack(contour_index=['y_ocean', 'x_ocean'])
    mask_y_numbered_1d = mask_y_numbered_1d.where(mask_y_numbered_1d > 0, drop=True)

    # Combine and sort contour
    contour_ordering = xr.concat((mask_x_numbered_1d, mask_y_numbered_1d), dim='contour_index')
    contour_ordering = contour_ordering.sortby(contour_ordering)
    contour_index_array = np.arange(1, len(contour_ordering) + 1)

    # Get lat/lon along contour
    lat_along_contour = contour_ordering.y_ocean
    lon_along_contour = contour_ordering.x_ocean
    lat_along_contour.coords['contour_index'] = contour_index_array
    lon_along_contour.coords['contour_index'] = contour_index_array

    # Save results
    contour_data[res] = {
        'lat': lat_along_contour,
        'lon': lon_along_contour,
        'contour_index': contour_index_array
    }
